<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/qiskit_kernel_sweep_full_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qiskit scikit-learn numpy matplotlib

In [ ]:
#!/usr/bin/env python3
"""
qiskit_kernel_sweep_full.py

Extended fidelity-kernel sweep with:
  - True class imbalances (including 10% and 20% minority classes)
  - Multiple feature maps (ZZFeatureMap, PauliFeatureMap)
  - SVM CV with and without class weighting
  - Kernel-target alignment (full and mini-batch)
  - Fidelity² histograms for same-class vs cross-class
"""

import warnings
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from qiskit.circuit.library import zz_feature_map, PauliFeatureMap
from qiskit.quantum_info import Statevector, state_fidelity

# Suppress Qiskit deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def generate_data(n_samples, imbalance, random_state=None):
    """Generate 2D binary classification data with specified class imbalance."""
    if imbalance >= 1.0:
        weights = [0.5, 0.5]
    else:
        weights = [imbalance, 1.0 - imbalance]
    X, y = make_classification(
        n_samples=n_samples,
        n_features=2,
        n_redundant=0,
        n_clusters_per_class=1,
        weights=weights,
        class_sep=2.0,
        random_state=random_state
    )
    return X, y

def build_feature_map(name, reps):
    """Return the specified feature map circuit with given repetition depth."""
    if name == "ZZ":
        return zz_feature_map(
            feature_dimension=2,
            reps=reps,
            entanglement='circular',
            insert_barriers=False
        )
    elif name == "Pauli":
        return PauliFeatureMap(
            feature_dimension=2,
            reps=reps,
            paulis=['X','Y','Z'],
            entanglement='circular',
            insert_barriers=False
        )
    else:
        raise ValueError(f"Unknown feature map: {name}")

def compute_fidelity_kernel(fmap, X):
    """Compute fidelity² Gram matrix for input data X using feature map fmap."""
    n = len(X)
    state_list = [Statevector.from_instruction(fmap.assign_parameters(x)) for x in X]
    K = np.zeros((n, n), float)
    for i in range(n):
        for j in range(i, n):
            f2 = state_fidelity(state_list[i], state_list[j])**2
            K[i, j] = K[j, i] = f2
    return K

def center_kernel(K):
    """Center kernel matrix K in feature (Hilbert) space."""
    n = K.shape[0]
    one_n = np.ones((n, n)) / n
    return K - one_n @ K - K @ one_n + one_n @ K @ one_n

def kernel_target_alignment(Kc, y):
    """Compute full kernel-target alignment."""
    yy = np.outer(y, y)
    numerator = np.trace(Kc @ yy)
    denominator = np.linalg.norm(Kc, 'fro') * np.linalg.norm(yy, 'fro')
    return numerator / denominator if denominator > 0 else 0.0

def mini_batch_alignment(K, y, batch_size=20, n_batches=50, random_state=None):
    """Estimate kernel-target alignment via random mini-batches."""
    rng = np.random.default_rng(random_state)
    y_signed = 2*y - 1  # map {0,1} → {−1,+1}
    align_vals = []
    n = len(y)
    for _ in range(n_batches):
        idx = rng.choice(n, size=min(batch_size, n), replace=False)
        Kb = K[np.ix_(idx, idx)]
        yb = y_signed[idx]
        yyb = np.outer(yb, yb)
        num = np.trace(Kb @ yyb)
        den = np.linalg.norm(Kb, 'fro') * np.linalg.norm(yyb, 'fro')
        align_vals.append(num/den if den > 0 else 0.0)
    return np.mean(align_vals)

def svm_cv(K, y, weighted=False, max_splits=5):
    """Perform stratified CV for SVC with precomputed kernel."""
    classes, counts = np.unique(y, return_counts=True)
    n_splits = min(max_splits, counts.min())
    if len(classes) < 2 or n_splits < 2:
        return np.array([])
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    svc = SVC(kernel='precomputed', class_weight='balanced' if weighted else None)
    return cross_val_score(svc, K, y, cv=cv)

def plot_fidelity_histograms(K, y, title=None, bins=30):
    """Plot fidelity² distributions for same-class vs cross-class pairs."""
    idx = np.triu_indices_from(K, k=1)
    vals = K[idx]
    yy = np.outer(y, y)
    same_mask = yy[idx] == 1
    same_vals  = vals[same_mask]
    cross_vals = vals[~same_mask]
    plt.figure(figsize=(6, 4))
    plt.hist(same_vals,  bins=bins, alpha=0.6, label='same-class')
    plt.hist(cross_vals, bins=bins, alpha=0.6, label='cross-class')
    plt.xlabel('Fidelity²')
    plt.ylabel('Count')
    if title:
        plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

def main():
    np.random.seed(42)
    n_samples         = 50
    imbalance_levels  = [1.0, 0.75, 0.5, 0.2, 0.1]
    feature_maps      = ["ZZ", "Pauli"]
    depths            = [1, 3, 5]

    for imb in imbalance_levels:
        X, y = generate_data(n_samples, imb, random_state=1)
        print(f"\n=== Imbalance {int(imb * 100)}% (N={n_samples}) ===")
        for name in feature_maps:
            for d in depths:
                fmap     = build_feature_map(name, d)
                K        = compute_fidelity_kernel(fmap, X)
                Kc       = center_kernel(K)
                full_al  = kernel_target_alignment(Kc, y)
                mb_al    = mini_batch_alignment(K, y,
                                                batch_size=25,
                                                n_batches=100,
                                                random_state=0)
                scores   = svm_cv(K, y, weighted=False)
                scores_b = svm_cv(K, y, weighted=True)
                fmt = lambda s: f"{s.mean():.3f}±{s.std():.3f}" if s.size>0 else "n/a"
                print(f"{name:6s} depth={d} | full_align={full_al:.3f} "
                      f"| mb_align={mb_al:.3f} "
                      f"| SVM={fmt(scores)} | SVM_bal={fmt(scores_b)}")
                plot_fidelity_histograms(
                    K, y,
                    title=f"{name} d={d}, Imb={int(imb*100)}%"
                )

if __name__ == "__main__":
    main()